In [1]:
import sys
import time
sys.path.insert(0, '/home/anish/ASC_ML')

In [2]:
from ASC_ML import multiple_model_gen_v2 as multiple
from ASC_ML import dataframe_extractor as de
from ASC_ML import model_generation as model_gen
# from ASC_ML import model_gen_train_test as testing
from dask_ml.preprocessing import LabelEncoder
# from dask_ml.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

2022-06-19 22:26:31.210200: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
# m = multiple.Multiple_Model_Gen(max_no_layers = 2, model_per_batch = 10)

In [4]:
# import dask.dataframe as dd
# import pandas as pd
# data = dd.from_pandas(pd.Series(['a', 'a', 'b'], dtype='category'),
#                      npartitions=1)
# le = LabelEncoder()
# le.fit_transform(data).compute()
# print(type(le.fit_transform(data).compute()))

In [5]:
#For House Prediction Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv
directory1 = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv"


# returns list of dask dataframe [singular_df_x, singular_df_y] or [train_df_x, train_df_y, test_df_x, test_df_y]
dataset_list = de.DataframeExtractor_csv(directory1, label_names = ["SalePrice"]).df_list

print("Train Dataset X Columns")
dataset_list[0].head()

Reading single csv from /home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv
Train Dataset X Columns


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,12.0,2008.0,WD,Normal


In [6]:
dataset_list[1].head()

,SalePrice
0,208500.0
1,181500.0
2,223500.0
3,140000.0
4,250000.0


In [7]:
train_x = dataset_list[0].copy()

In [8]:
train_x = train_x.fillna(0)

In [9]:
train_x.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [10]:
train_x['MSSubClass'] = train_x['MSSubClass'].apply(str)

train_x['OverallCond'] = train_x['OverallCond'].astype(str)

train_x['YrSold'] = train_x['YrSold'].astype(str)
train_x['MoSold'] = train_x['MoSold'].astype(str)
train_x.head()

/home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/dask/dataframe/core.py:3946: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('MSSubClass', 'object'))

  warnings.warn(meta_warning(meta))


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [11]:
# a = train_x['LotShape']

In [12]:
# le = LabelEncoder() 
# le.fit_transform(a).compute()
# train_x['LotShape'] = le.fit_transform(train_x['LotShape'])

In [13]:
# train_x.head()

In [14]:
def keyval(allkeys):
    i = 0
    keyvalpairs = list()
    for key in allkeys:
        keyvalpairs.append((key,i))
        i+=1
    return dict(keyvalpairs)

In [15]:
# cols = ['FireplaceQu', 'LotShape']
cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold']
keys = {}
for col in cols:
    key = train_x[col].unique()
    keyvalpairs = keyval(key)
    keys.update({col:keyvalpairs})


In [16]:
keys

{'FireplaceQu': {0: 0, 'TA': 1, 'Gd': 2, 'Fa': 3, 'Ex': 4, 'Po': 5},
 'BsmtQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 0: 3, 'Fa': 4},
 'BsmtCond': {'TA': 0, 'Gd': 1, 0: 2, 'Fa': 3, 'Po': 4},
 'GarageQual': {'TA': 0, 'Fa': 1, 'Gd': 2, 0: 3, 'Ex': 4, 'Po': 5},
 'GarageCond': {'TA': 0, 'Fa': 1, 0: 2, 'Gd': 3, 'Po': 4, 'Ex': 5},
 'ExterQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'ExterCond': {'TA': 0, 'Gd': 1, 'Fa': 2, 'Po': 3, 'Ex': 4},
 'HeatingQC': {'Ex': 0, 'Gd': 1, 'TA': 2, 'Fa': 3, 'Po': 4},
 'PoolQC': {0: 0, 'Ex': 1, 'Fa': 2, 'Gd': 3},
 'KitchenQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'BsmtFinType1': {'GLQ': 0,
  'ALQ': 1,
  'Unf': 2,
  'Rec': 3,
  'BLQ': 4,
  0: 5,
  'LwQ': 6},
 'BsmtFinType2': {'Unf': 0,
  'BLQ': 1,
  0: 2,
  'ALQ': 3,
  'Rec': 4,
  'LwQ': 5,
  'GLQ': 6},
 'Functional': {'Typ': 0,
  'Min1': 1,
  'Maj1': 2,
  'Min2': 3,
  'Mod': 4,
  'Maj2': 5,
  'Sev': 6},
 'Fence': {0: 0, 'MnPrv': 1, 'GdWo': 2, 'GdPrv': 3, 'MnWw': 4},
 'BsmtExposure': {'No': 0, 'Gd': 1, 'Mn': 2, '

In [17]:
train_X = train_x.replace(keys).copy()

In [18]:
train_X['FireplaceQu'].unique().compute()

0    0
1    1
2    2
3    3
4    4
5    5
Name: FireplaceQu, dtype: int64

In [19]:
train_X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,0,RL,65.0,8450.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,0,WD,Normal
1,2.0,1,RL,80.0,9600.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,1,1,WD,Normal
2,3.0,0,RL,68.0,11250.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2,0,WD,Normal
3,4.0,2,RL,60.0,9550.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,2,WD,Abnorml
4,5.0,0,RL,84.0,14260.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,3,0,WD,Normal


In [20]:
cols_to_remove = []
for col in train_X.columns:
#     print(type(col))
    try:
        a = train_X[col].astype(float).compute()
    except ValueError:
#         print('Couldn\'t covert %s to float' % col)
        cols_to_remove.append(col)
        pass

# keep only the columns in df that do not contain string
train_X = train_X[[col for col in train_X.columns if col not in cols_to_remove]]

In [21]:
print(cols_to_remove)

['MSZoning', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']


In [22]:
train_X = train_X.drop(["Id"], axis = 1)

In [23]:
# tsklearn.preprocessing.MinMaxScalerrain_X.head()

In [24]:
len(train_X.columns)

58

In [25]:
len(dataset_list[1].columns)

1

In [26]:
scaler_x = MinMaxScaler()
scaler_x.fit(train_X)
train_X_scaled = scaler_x.transform(train_X)

scaler_y = MinMaxScaler()
scaler_y.fit(dataset_list[1])
train_Y_scaled = scaler_y.transform(dataset_list[1])

In [27]:
# col = train_X.columns.to_list()
# scaler_d = MinMaxScaler()
# train_X.columns = list(train_X.columns)
# scaled_train_X = scaler_d.fit_transform(train_X[col])

In [28]:
train_X_scaled
# train_Y_scaled

array([[0.        , 0.20766773, 0.0334198 , ..., 0.        , 0.        ,
        0.        ],
       [0.07142857, 0.25559105, 0.03879502, ..., 0.        , 0.09090909,
        0.25      ],
       [0.        , 0.2172524 , 0.04650728, ..., 0.        , 0.18181818,
        0.        ],
       ...,
       [0.14285714, 0.21086262, 0.03618687, ..., 0.16129032, 0.09090909,
        1.        ],
       [0.07142857, 0.2172524 , 0.03934189, ..., 0.        , 0.63636364,
        1.        ],
       [0.07142857, 0.23961661, 0.04037019, ..., 0.        , 1.        ,
        0.        ]])

In [29]:
train_Y_scaled

array([[0.24107763],
       [0.20358284],
       [0.26190807],
       ...,
       [0.321622  ],
       [0.14890293],
       [0.15636717]])

In [30]:
m = multiple.Multiple_Model_Gen_V2(train_X_scaled, train_Y_scaled, 50, 128, input_shape = 58, model_per_batch = 10)

In [31]:
m.get_model_confs()

[[(16, 16), (16, 32), (16, 64), (16, 128), (16, 196), (16, 256), (32, 16), (32, 32), (32, 64), (32, 128), (32, 196), (32, 256), (64, 16), (64, 32), (64, 64), (64, 128), (64, 196), (64, 256), (128, 16), (128, 32), (128, 64), (128, 128), (128, 196), (128, 256), (196, 16), (196, 32), (196, 64), (196, 128), (196, 196), (196, 256), (256, 16), (256, 32), (256, 64), (256, 128), (256, 196), (256, 256)], [(16, 16, 16), (16, 16, 32), (16, 16, 64), (16, 16, 128), (16, 16, 196), (16, 16, 256), (16, 32, 16), (16, 32, 32), (16, 32, 64), (16, 32, 128), (16, 32, 196), (16, 32, 256), (16, 64, 16), (16, 64, 32), (16, 64, 64), (16, 64, 128), (16, 64, 196), (16, 64, 256), (16, 128, 16), (16, 128, 32), (16, 128, 64), (16, 128, 128), (16, 128, 196), (16, 128, 256), (16, 196, 16), (16, 196, 32), (16, 196, 64), (16, 196, 128), (16, 196, 196), (16, 196, 256), (16, 256, 16), (16, 256, 32), (16, 256, 64), (16, 256, 128), (16, 256, 196), (16, 256, 256), (32, 16, 16), (32, 16, 32), (32, 16, 64), (32, 16, 128), (32

In [32]:
m.model_confs

[[['', 58, 2, 'relu', {'layer1': 16, 'layer2': 16}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 16, 'layer2': 32}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 16, 'layer2': 64}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 16, 'layer2': 128}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 16, 'layer2': 196}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 16, 'layer2': 256}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 16}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 32}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 64}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 128}, [1, None]]],
 [['', 58, 2, 'relu', {'layer1': 32, 'layer2': 196}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 256}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 64, 'layer2': 16}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 64, 'layer2': 32}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 64, 'layer2': 64}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 

In [33]:
# m.get_all_models()
m.train_models()

2022-06-19 22:26:43.330275: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-19 22:26:43.330947: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-19 22:26:43.409110: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-19 22:26:43.409374: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.695GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2022-06-19 22:26:43.409407: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-19 22:26:43.411727: I tensorflow/stream_executor/platform/de

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_dense_16_16 (InputL [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_16_32 (InputL [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_16_64 (InputL [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_16_128 (Input [(None, 58)]         0                                            
______________________________________________________________________________________________

2022-06-19 22:26:44.082103: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-19 22:26:44.082507: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2899885000 Hz


 1/12 [=>............................] - ETA: 17s - loss: 1.1408 - output_layer_dense_16_16_loss: 0.0933 - output_layer_dense_16_32_loss: 0.0143 - output_layer_dense_16_64_loss: 0.2363 - output_layer_dense_16_128_loss: 0.2827 - output_layer_dense_16_196_loss: 0.0625 - output_layer_dense_16_256_loss: 0.0474 - output_layer_dense_32_16_loss: 0.0257 - output_layer_dense_32_32_loss: 0.3185 - output_layer_dense_32_64_loss: 0.0478 - output_layer_dense_32_128_loss: 0.0123

2022-06-19 22:26:45.516399: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


...

12/12 [==============================] - 0s 6ms/step - loss: 0.0029 - output_layer_dense_256_128_64_loss: 3.7624e-04 - output_layer_dense_256_128_128_loss: 2.8949e-04 - output_layer_dense_256_128_196_loss: 2.3777e-04 - output_layer_dense_256_128_256_loss: 2.3059e-04 - output_layer_dense_256_196_16_loss: 3.1290e-04 - output_layer_dense_256_196_32_loss: 5.9000e-04 - output_layer_dense_256_196_64_loss: 2.5126e-04 - output_layer_dense_256_196_128_loss: 2.6992e-04 - output_layer_dense_256_196_196_loss: 2.1238e-04 - output_layer_dense_256_196_256_loss: 1.6015e-04
Epoch 30/50
12/12 [==============================] - 0s 5ms/step - loss: 0.0027 - output_layer_dense_256_128_64_loss: 3.0005e-04 - output_layer_dense_256_128_128_loss: 2.4675e-04 - output_layer_dense_256_128_196_loss: 2.1834e-04 - output_layer_dense_256_128_256_loss: 2.5497e-04 - output_layer_dense_256_196_16_loss: 2.7349e-04 - output_layer_dense_256_196_32_loss: 5.1493e-04 - output_layer_dense_256_196_64_loss: 2.4037e-04 - output_l

12/12 [==============================] - 0s 5ms/step - loss: 0.0015 - output_layer_dense_256_128_64_loss: 1.5789e-04 - output_layer_dense_256_128_128_loss: 9.2131e-05 - output_layer_dense_256_128_196_loss: 1.1573e-04 - output_layer_dense_256_128_256_loss: 1.7796e-04 - output_layer_dense_256_196_16_loss: 1.6069e-04 - output_layer_dense_256_196_32_loss: 3.3602e-04 - output_layer_dense_256_196_64_loss: 8.6749e-05 - output_layer_dense_256_196_128_loss: 1.2448e-04 - output_layer_dense_256_196_196_loss: 1.3332e-04 - output_layer_dense_256_196_256_loss: 1.6349e-04
Epoch 44/50
12/12 [==============================] - 0s 5ms/step - loss: 0.0014 - output_layer_dense_256_128_64_loss: 1.5066e-04 - output_layer_dense_256_128_128_loss: 1.2906e-04 - output_layer_dense_256_128_196_loss: 9.0896e-05 - output_layer_dense_256_128_256_loss: 1.9023e-04 - output_layer_dense_256_196_16_loss: 1.2171e-04 - output_layer_dense_256_196_32_loss: 2.9613e-04 - output_layer_dense_256_196_64_loss: 9.8841e-05 - output_l

12/12 [==============================] - 1s 4ms/step - loss: 0.2005 - output_layer_dense_256_256_16_loss: 0.0296 - output_layer_dense_256_256_32_loss: 0.0365 - output_layer_dense_256_256_64_loss: 0.0324 - output_layer_dense_256_256_128_loss: 0.0368 - output_layer_dense_256_256_196_loss: 0.0354 - output_layer_dense_256_256_256_loss: 0.0297
Epoch 2/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0383 - output_layer_dense_256_256_16_loss: 0.0059 - output_layer_dense_256_256_32_loss: 0.0062 - output_layer_dense_256_256_64_loss: 0.0068 - output_layer_dense_256_256_128_loss: 0.0066 - output_layer_dense_256_256_196_loss: 0.0069 - output_layer_dense_256_256_256_loss: 0.0060
Epoch 3/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0184 - output_layer_dense_256_256_16_loss: 0.0031 - output_layer_dense_256_256_32_loss: 0.0029 - output_layer_dense_256_256_64_loss: 0.0027 - output_layer_dense_256_256_128_loss: 0.0036 - output_layer_dense_256_256_196_loss: 0.0031 -

12/12 [==============================] - 0s 5ms/step - loss: 0.0025 - output_layer_dense_256_256_16_loss: 5.2778e-04 - output_layer_dense_256_256_32_loss: 5.8942e-04 - output_layer_dense_256_256_64_loss: 3.0210e-04 - output_layer_dense_256_256_128_loss: 4.4079e-04 - output_layer_dense_256_256_196_loss: 2.7339e-04 - output_layer_dense_256_256_256_loss: 3.2963e-04
Epoch 24/50
12/12 [==============================] - 0s 4ms/step - loss: 0.0023 - output_layer_dense_256_256_16_loss: 4.5481e-04 - output_layer_dense_256_256_32_loss: 5.3819e-04 - output_layer_dense_256_256_64_loss: 3.2689e-04 - output_layer_dense_256_256_128_loss: 4.0433e-04 - output_layer_dense_256_256_196_loss: 2.9580e-04 - output_layer_dense_256_256_256_loss: 2.8670e-04
Epoch 25/50
12/12 [==============================] - 0s 5ms/step - loss: 0.0020 - output_layer_dense_256_256_16_loss: 4.1497e-04 - output_layer_dense_256_256_32_loss: 4.8622e-04 - output_layer_dense_256_256_64_loss: 2.7683e-04 - output_layer_dense_256_256_12

12/12 [==============================] - 0s 4ms/step - loss: 7.5477e-04 - output_layer_dense_256_256_16_loss: 1.9219e-04 - output_layer_dense_256_256_32_loss: 1.6590e-04 - output_layer_dense_256_256_64_loss: 8.8648e-05 - output_layer_dense_256_256_128_loss: 1.1875e-04 - output_layer_dense_256_256_196_loss: 1.1606e-04 - output_layer_dense_256_256_256_loss: 7.3217e-05
Epoch 45/50
12/12 [==============================] - 0s 4ms/step - loss: 6.5391e-04 - output_layer_dense_256_256_16_loss: 1.5290e-04 - output_layer_dense_256_256_32_loss: 1.4891e-04 - output_layer_dense_256_256_64_loss: 9.4971e-05 - output_layer_dense_256_256_128_loss: 9.8650e-05 - output_layer_dense_256_256_196_loss: 9.7505e-05 - output_layer_dense_256_256_256_loss: 6.0975e-05
Epoch 46/50
12/12 [==============================] - 0s 4ms/step - loss: 6.2758e-04 - output_layer_dense_256_256_16_loss: 1.5768e-04 - output_layer_dense_256_256_32_loss: 1.4542e-04 - output_layer_dense_256_256_64_loss: 7.2390e-05 - output_layer_dens

In [34]:
m.evaluate_dict_list

[{'model_name': 'dense_196_196_64',
  'score': 3.7397014239104465e-05,
  'path_weights': ''},
 {'model_name': 'dense_256_196_64',
  'score': 5.097092071082443e-05,
  'path_weights': ''},
 {'model_name': 'dense_196_256_128',
  'score': 5.181122469366528e-05,
  'path_weights': ''},
 {'model_name': 'dense_196_256_256',
  'score': 5.482707274495624e-05,
  'path_weights': ''},
 {'model_name': 'dense_196_196_196',
  'score': 5.7864479458658025e-05,
  'path_weights': ''},
 {'model_name': 'dense_196_256_196',
  'score': 5.8861107390839607e-05,
  'path_weights': ''},
 {'model_name': 'dense_196_128_196',
  'score': 6.520036549773067e-05,
  'path_weights': ''},
 {'model_name': 'dense_196_196_256',
  'score': 6.626684626098722e-05,
  'path_weights': ''},
 {'model_name': 'dense_256_256_64',
  'score': 6.656890036538243e-05,
  'path_weights': ''},
 {'model_name': 'dense_256_196_196',
  'score': 6.826928438385949e-05,
  'path_weights': ''}]